<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [1]:
using DataFrames,CSV,JWAS,JWAS.Datasets

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### pedigree

In [4]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [5]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

In [6]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [8]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [9]:
model    = build_model(model_equations,R);

In [10]:
set_covariate(model,"age");

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [11]:
ped = get_pedigree(pedfile);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:01

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [12]:
set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [14]:
out = runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                         10
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





running MCMC for conventional (no markers)...  0%|      |  ETA: 0:07:47


Posterior means at iteration: 500
Residual covariance matrix: 


running MCMC for conventional (no markers)... 25%|█     |  ETA: 0:00:02

[104.124 26.2121; 26.2121 7.32556]
Polygenic effects covariance matrix 
[33.2182 4.31627; 4.31627 2.43989]



running MCMC for conventional (no markers)... 50%|███   |  ETA: 0:00:01


Posterior means at iteration: 1000
Residual covariance matrix: 
[81.3315 20.3175; 20.3175 5.78435]
Polygenic effects covariance matrix 
[53.594 9.25546; 9.25546 3.74766]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


In [15]:
out["MCMC samples for residual covariance matrix"]

4×100 Array{Float64,2}:
 117.321    44.3716   90.1009   90.132    …  84.898    110.494    82.4971 
  19.8691    7.04933  23.8387   14.1878      19.2768    20.4771   20.563  
  19.8691    7.04933  23.8387   14.1878      19.2768    20.4771   20.563  
   3.59992   1.40274   6.57932   2.65989      4.76932    4.13422   5.29953

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [16]:
phenofile = Datasets.dataset("testMT","maternal.txt")
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [17]:
model_equations = "BW = intercept + age + sex + Animal+ dam;
                   CW = intercept + age + sex + Animal";

In [18]:
model = build_model(model_equations,R);
set_covariate(model,"age");

In [19]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
set_random(model,"Animal dam", ped,G0)

INFO: dam is not found in model equation 2.


In [21]:
out=runMCMC(model,data,chain_length=1000,printout_frequency=500);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000




Posterior means at iteration: 500
Residual covariance matrix: 
[150.715 34.6499; 34.6499 8.54643]
Polygenic effects covariance matrix 
[5.23974 1.01862 0.159582; 1.01862 0.907634 -0.037244; 0.159582 -0.037244 0.651138]



running MCMC for conventional (no markers)... 20%|█     |  ETA: 0:00:00


Posterior means at iteration: 1000
Residual covariance matrix: 
[161.705 37.4924; 37.4924 9.2784]
Polygenic effects covariance matrix 
[4.6876 0.922553 0.112737; 0.922553 0.865759 -0.026022; 0.112737 -0.026022 0.537082]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:00


In [22]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 3 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of location parameters"